In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import string

# IMSDB

In [30]:
import os

movie_url = 'http://www.imsdb.com/alphabetical/'
script_url = 'http://www.imsdb.com/scripts/'

if not os.path.exists('../data/raw_data'):
    os.makedirs('../data/raw_data')

def get_movies_on_page(soup):
    
    content =  soup.find_all('td', {'valign': 'top'})[2]
    movie_names = [p.a.get('title').replace('Script', '').strip() for p in content.find_all('p')]
    return movie_names

for ch in string.ascii_uppercase:
    url = movie_url + ch
    page = urllib.urlopen(url)
    soup = BeautifulSoup(page, 'html')
    movie_names = get_movies_on_page(soup)
    dir = '../data/raw_data' + ch
    if not os.path.exists(dir):
        os.makedirs(dir)
    else:
        print "skipping"
        continue
    
    for mv_name in movie_names:
        url = script_url + mv_name.replace(' ', '-') + '.html'
        
        page = urllib.urlopen(url)
        soup = BeautifulSoup(page, 'html')
        content =  soup.find_all('td', {'valign': 'top'})[2]
        with open(dir + '/' + mv_name + '.txt', 'w') as f:
            print "Writing:", mv_name
            f.write(str(content))
    break

Writing: A Few Good Men
Writing: A Most Violent Year
Writing: A Serious Man
Writing: A.I.
Writing: Above the Law
Writing: Absolute Power
Writing: Abyss, The
Writing: Ace Ventura: Pet Detective
Writing: Adaptation
Writing: Addams Family, The
Writing: Adjustment Bureau, The
Writing: Adventures of Buckaroo Banzai Across the Eighth Dimension, The
Writing: Affliction
Writing: After School Special
Writing: After.Life
Writing: Agnes of God
Writing: Air Force One
Writing: Airplane
Writing: Airplane 2: The Sequel
Writing: Aladdin
Writing: Ali
Writing: Alien
Writing: Alien 3
Writing: Alien Nation
Writing: Alien vs. Predator
Writing: Alien: Resurrection
Writing: Aliens
Writing: All About Eve
Writing: All About Steve
Writing: All the King's Men
Writing: All the President's Men
Writing: Almost Famous
Writing: Alone in the Dark
Writing: Amadeus
Writing: Amelia
Writing: American Beauty
Writing: American Gangster
Writing: American Graffiti
Writing: American History X
Writing: American Hustle
Writing: 

# Scraping friends

In [86]:
URL = 'http://www.livesinabox.com/friends/'
script_url = URL + 'scripts.shtml'
page = urllib.urlopen(script_url)
soup = BeautifulSoup(page)
links = [link.a.get('href') for link in soup.find_all('li')]

def get_dialog(string):
    
    dialog = ''
    try:
        string = str(string.text)
        if len(string) > 0 and string[0].isalpha() and string.index(':') < 10:
            dialog = string
    except:
        pass
    return dialog

i = 0
for link in links:
    url = URL + link
    page = urllib.urlopen(url)
    soup = BeautifulSoup(page)
    dialogs = [get_dialog(p) for p in soup.find_all('p')]
    dialogs = [d for d in dialogs if len(d) > 0]
    
    if not os.path.exists('../data/raw_data/friends'):
        os.makedirs('../data/raw_data/friends')
    with open('../data/raw_data/friends/' + str(i) + '.txt', 'w') as f:
        print "Wrote:", i
        f.write('\n'.join(dialogs))
        i += 1

Wrote: 0
Wrote: 1
Wrote: 2
Wrote: 3
Wrote: 4
Wrote: 5
Wrote: 6
Wrote: 7
Wrote: 8
Wrote: 9
Wrote: 10
Wrote: 11
Wrote: 12
Wrote: 13
Wrote: 14
Wrote: 15
Wrote: 16
Wrote: 17
Wrote: 18
Wrote: 19
Wrote: 20
Wrote: 21
Wrote: 22
Wrote: 23
Wrote: 24
Wrote: 25
Wrote: 26
Wrote: 27
Wrote: 28
Wrote: 29
Wrote: 30
Wrote: 31
Wrote: 32
Wrote: 33
Wrote: 34
Wrote: 35
Wrote: 36
Wrote: 37
Wrote: 38
Wrote: 39
Wrote: 40
Wrote: 41
Wrote: 42
Wrote: 43
Wrote: 44
Wrote: 45
Wrote: 46
Wrote: 47
Wrote: 48
Wrote: 49
Wrote: 50
Wrote: 51
Wrote: 52
Wrote: 53
Wrote: 54
Wrote: 55
Wrote: 56
Wrote: 57
Wrote: 58
Wrote: 59
Wrote: 60
Wrote: 61
Wrote: 62
Wrote: 63
Wrote: 64
Wrote: 65
Wrote: 66
Wrote: 67
Wrote: 68
Wrote: 69
Wrote: 70
Wrote: 71
Wrote: 72
Wrote: 73
Wrote: 74
Wrote: 75
Wrote: 76
Wrote: 77
Wrote: 78
Wrote: 79
Wrote: 80
Wrote: 81
Wrote: 82
Wrote: 83
Wrote: 84
Wrote: 85
Wrote: 86
Wrote: 87
Wrote: 88
Wrote: 89
Wrote: 90
Wrote: 91
Wrote: 92
Wrote: 93
Wrote: 94
Wrote: 95
Wrote: 96
Wrote: 97
Wrote: 98
Wrote: 99
Wrote: 100

# Cleaning Data

## Movies data

In [8]:
import glob
import os

if not os.path.exists('../data/clean_data'):
    os.makedirs('../data/clean_data')

def clean_html(line):
    text = BeautifulSoup(line).text
    if '*' in text:
        text = text.replace('*', '')
    if '(' in text:
        text = text.replace('(', '( ')
    if ')' in text:
        text = text.replace(')', ' )')        
    if '...' in text:
        text = text.replace('...', '.') 
    if '--' in text:
        text = text.replace('--', '') 
    return text

def left_space_count(line):

    if len(line) == 0:
        return 0
    if line[0] == '<':
        line = clean_html(line)
    line = line.replace('\t', ' ')
    count = 0
    for c in line:
        if c == ' ':
            count += 1
        else:
            break
    return count
        
def is_character_name(line):
    if clean_html(line).isupper():
        if 12 <= left_space_count(line) <= 38:
            return True
    else:
        return False

def in_dialog(line):
    return 8 <= left_space_count(line) <= 20

def write_to_file(fname, char, string):
    dir = '../data/clean_data/' + char + '/'
    if not os.path.exists(dir):
        os.makedirs(dir)
    with open('../data/clean_data/' + fname.split('/')[-2] + '/' + fname.split('/')[-1], 'w') as f:
        try:
            
            f.write(string)
            fno += 1
        except:
            print "Skipped"


def clean_direc(starts_with):
    
#     if os.path.exists('clean_data/' + starts_with):
#         print "Already cleaned, skipping this"
#         return
    fno = 0                    
    fnames = [f for f in glob.iglob('../data/raw_data/' + starts_with + '/*.txt')]
    while fno < len(fnames):
        fname = fnames[fno]
        fno += 1
        print fname
        string = fname + '\n'
        lines = [line.replace('\t', ' ') for line in open(fname).readlines() if len(line.strip()) > 0]

        i = 0
        characters = []
        dialogs = []
        string = ""
        while i < len(lines):
            line = lines[i]
            
            #Check if it qualifies as a character name
            if line.strip()[:2] == '<b' and 12 <= left_space_count(line) <= 38:
                character = clean_html(line).strip()
                start = i + 1
                dialog = ""
                while start < len(lines):
                    d = lines[start]
                    if d.strip()[:2] == '<b':
                        break
                    # Check if it qualifies as a dialog
                    if 10 <= left_space_count(d) <= 40:
                        dialog += clean_html(d).strip() + ' '
                    start += 1

                char = character.strip()
                dial = dialog.strip()
                if len(char) > 0 and len(dial) > 0:
    #                 print char + ":" + dial
                    string += char + ": " + dial + '\n'
                    characters.append(char)
                    dialogs.append(dial)
            i += 1
        if len(dialogs) > 100:
            print "Writing:", fno, "Dialogs:", len(dialogs)
            write_to_file(fname, starts_with, string)

import string
for starts_with in string.ascii_uppercase:
    clean_direc(starts_with)

../data/raw_data/A/A Few Good Men.txt
Writing: 1 Dialogs: 1537
Skipped
../data/raw_data/A/A Most Violent Year.txt
Writing: 2 Dialogs: 1062
Skipped
../data/raw_data/A/A Serious Man.txt
../data/raw_data/A/A.I..txt
../data/raw_data/A/Above the Law.txt
Writing: 5 Dialogs: 577
Skipped
../data/raw_data/A/Absolute Power.txt
Writing: 6 Dialogs: 514
Skipped
../data/raw_data/A/Abyss, The.txt
Writing: 7 Dialogs: 689
Skipped
../data/raw_data/A/Ace Ventura: Pet Detective.txt
../data/raw_data/A/Adaptation.txt
Writing: 9 Dialogs: 818
Skipped
../data/raw_data/A/Addams Family, The.txt
Writing: 10 Dialogs: 278
Skipped
../data/raw_data/A/Adjustment Bureau, The.txt
Writing: 11 Dialogs: 965
Skipped
../data/raw_data/A/Adventures of Buckaroo Banzai Across the Eighth Dimension, The.txt
../data/raw_data/A/Affliction.txt
Writing: 13 Dialogs: 849
Skipped
../data/raw_data/A/After School Special.txt
Writing: 14 Dialogs: 1184
Skipped
../data/raw_data/A/After.Life.txt
../data/raw_data/A/Agnes of God.txt
Writing: 16 

KeyboardInterrupt: 